## 不一致的预处理

比如，特征预处理、降维等处理

如果这些数据转换是在训练模型时使用的，那么它们也必须用于后续的数据集（测试集）。否则，特征空间将发生变化，模型将无法有效地执行。

In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

random_state = 42
X, y = make_regression(random_state=random_state, n_features=1, noise=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=random_state)

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# 注意，转换必须只在训练集上学习！！
# 错误： 标准化只在训练集上执行，没在测试集上执行
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
model = LinearRegression().fit(X_train_transformed, y_train)
mean_squared_error(y_test, model.predict(X_test))

62.80867119249524

In [4]:
# 正确：标准化也必须在测试集上进行转换，这样模型拟合的效果才好

X_test_transformed = scaler.transform(X_test)
mean_squared_error(y_test, model.predict(X_test_transformed))

0.9027975466369481

使用管道pipeline，可以更容易地将转换与估计器连接起来处理，这样不会忘记测试集的转换。

In [5]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(), LinearRegression())

model.fit(X_train, y_train)
mean_squared_error(y_test, model.predict(X_test))

0.9027975466369481

## 数据泄露

如果在建立模型时`使用了在预测时无法获得的信息，就会发生数据泄漏`。

这将导致过于乐观的性能估计，例如来自交叉验证，因此当模型用于实际的新数据时，性能较差。

一个常见的原因是：`没有把测试和训练数据子集分开`。测试数据不应该被用来对模型进行选择。一般的规则是，`永远不要在测试数据上调用拟合`。虽然这听起来很明显，但在某些情况下很容易错过，例如在应用某些预处理步骤时。

### 预处理

这种泄漏的风险与scikit-learn中`几乎所有的转换都有关`，包括（但不限于）`StandardScaler`、`SimpleImputer`和`PCA`。

In [7]:
import numpy as np
n_samples, n_features, n_classes = 200, 10000, 2
rng = np.random.RandomState(42)
X = rng.standard_normal((n_samples, n_features))
y = rng.choice(n_classes, n_samples)

由于特征选择步骤 "看到 "了测试数据，该模型有一个不公平的优势。

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# 错误处理，整个数据集被转换了。。。
X_selected = SelectKBest(k=25).fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, random_state=42)
gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train, y_train)

# 乐观估计了~
y_pred = gbc.predict(X_test)
accuracy_score(y_test, y_pred)

0.76

`为了防止数据泄露，好的做法是先将数据分成训练和测试子集`。然后，可以`只使用训练数据集来形成特征选择`。

注意，每当我们`使用fit或fit_transform`时，我们`只使用训练数据集`。

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
select = SelectKBest(k=25)

# 正确的转换，只用训练集去训练并进行转换
X_train_selected = select.fit_transform(X_train, y_train)

gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train_selected, y_train)

# 进行预测前，先对测试集进行转换
X_test_selected = select.transform(X_test)
y_pred = gbc.predict(X_test_selected)
accuracy_score(y_test, y_pred)

0.46

推荐：`管道pipeline确保在进行拟合时只使用训练数据`，而测试数据只用于计算准确性分数。

In [12]:
from sklearn.pipeline import make_pipeline

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

pipeline = make_pipeline(SelectKBest(k=25),
                         GradientBoostingClassifier(random_state=1))
pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.46